<div style="width:100%; background-color: #000041"><a target="_blank" href="http://university.yugabyte.com"><img src="assets/YBU_Logo.webp" /></a></div>

# YSQL Fundamentals



## Requirements
Before running the cells in this notebook, you must first execute all the cells in the following notebook:
- `01_Lab_Setup.ipynb`

## Getting help
The best way to get help from the Yugabyte University team is to post your question on YugabyteDB Community Slack in the #training or #yb-university channels. To sign up, visit [https://communityinviter.com/apps/yugabyte-db/register](https://communityinviter.com/apps/yugabyte-db/register).

## Connect to the `db_ybu` database
Run all the cells in this section:
- Connect using Python and PostgreSQL driver
- Load the SQL magic extension
- Create the prepared statements

In [ ]:
# connect use Python 3.7.9+
import psycopg2
import sqlalchemy as alc
from sqlalchemy import create_engine

# Inspiration from https://medium.com/analytics-vidhya/postgresql-integration-with-jupyter-notebook-deb97579a38d
# Use %store -r to read 01_Lab_Requirements_Setup variables

%store -r MY_DB_NAME
%store -r MY_YB_PATH
%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02
%store -r MY_HOST_IPv4_03
%store -r MY_GITPOD_WORKSPACE_URL

%store -r MY_NOTEBOOK_DATA_FOLDER
%store -r MY_NOTEBOOK_UTILS_FOLDER

%store -r MY_DATA_DDL_FILE_01
%store -r MY_DATA_DML_FILE_01
%store -r MY_DATA_DDL_FILE_02
%store -r MY_DATA_DML_FILE_02
%store -r MY_DATA_DDL_FILE_03
%store -r MY_DATA_DML_FILE_03

db_host=MY_HOST_IPv4_01
db_name=MY_DB_NAME


connection_str='postgresql+psycopg2://yugabyte@'+db_host+':5433/'+db_name


# engine = create_engine(connection_str)

### Load the SQL magic extension

In [ ]:
%reload_ext sql

# SQL magic for python connection string
%sql {connection_str}

### Create the prepared statements

> IMPORTANT!
>   
> In order to create the prepared statements for the SQL magic connection, you must run the following cell!!!
> 
> Do not skip this step.
> 

In [ ]:
#%% python, but prepared statements as sql magic
if (MY_GITPOD_WORKSPACE_URL is None):
    a = %sql select fn_yb_create_stmts()
else:
    WORKSPACE_URL = MY_GITPOD_WORKSPACE_URL.replace('https://','https://7000-')
    a = %sql select fn_yb_create_stmts(:WORKSPACE_URL)

print (a)

Confirm that the following query returns a count of 3 (for three prepared statements).

In [ ]:
%%sql 
select count(*) from pg_prepared_statements where 1=1 and name in ('stmt_util_metrics_snap_tablet','stmt_util_metrics_snap_table','stmt_util_metrics_snap_reset')

---


## q1 | Basics of Data Definition Language (DDL) and Data Manipulation Language (DML)

<!--

insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7369, 'SMITH', 'CLERK', 7902, '1980-12-17', 800, NULL, 20,'SMITH@acme.com', '{"skills":["accounting"]}'), (7499, 'ALLEN', 'SALESMAN', 7698, '1981-02-20', 1600, 300, 30,'ALLEN@acme.com', null), (7521, 'WARD', 'SALESMAN', 7698, '1981-02-22', 1250, 500, 30,'WARD@compuserve.com', null), (7566, 'JONES', 'MANAGER', 7839, '1981-04-02', 2975, NULL, 20,'JONES@gmail.com', null), (7654, 'MARTIN', 'SALESMAN', 7698, '1981-09-28', 1250, 1400, 30,'MARTIN@acme.com', null), (7698, 'BLAKE', 'MANAGER', 7839, '1981-05-01', 2850, NULL, 30,'BLAKE@hotmail.com', null), (7782, 'CLARK', 'MANAGER', 7839, '1981-06-09', 2450, NULL, 10,'CLARK@acme.com', '{"skills":["C","C++","SQL"]}'), (7788, 'SCOTT', 'ANALYST', 7566, '1982-12-09', 3000, NULL, 20,'SCOTT@acme.com', '{"cat":"tiger"}'), (7839, 'KING', 'PRESIDENT', NULL, '1981-11-17', 5000, NULL, 10,'KING@aol.com', null), (7844, 'TURNER', 'SALESMAN', 7698, '1981-09-08', 1500, 0, 30,'TURNER@acme.com', null), (7876, 'ADAMS', 'CLERK', 7788, '1983-01-12', 1100, NULL, 20,'ADAMS@acme.org', null), (7900, 'JAMES', 'CLERK', 7698, '1981-12-03', 950, NULL, 30,'JAMES@acme.org', null), (7902, 'FORD', 'ANALYST', 7566, '1981-12-03', 3000, NULL, 20,'FORD@acme.com', '{"skills":["YSQL","YCQL","SQL","CQL"]}'), (7934, 'MILLER', 'CLERK', 7782, '1982-01-23', 1300, NULL, 10,'MILLER@acme.com', null);
>



### Scenario 1: Create a table and insert rows

Create two related tables, `tbl_departments` and `tbl_employees`:

In [ ]:
%%sql

drop table if exists tbl_departments;

create table if not exists tbl_departments (
  deptno integer NOT NULL,
  dname text,
  loc text,
  description text,
  CONSTRAINT pk_dept PRIMARY KEY (deptno asc)
);

drop table if exists tbl_employees;

create table if not exists tbl_employees (
  empno integer generated by default as identity (start with 10000) NOT NULL,
  ename text NOT NULL,
  job text,
  mgr integer,
  hiredate date,
  sal integer,
  comm integer,
  deptno integer NOT NULL,
  email text,
  other_info jsonb,
  CONSTRAINT pk_emp PRIMARY KEY (empno hash),
  CONSTRAINT emp_email_uk UNIQUE (email),
  CONSTRAINT emp_email_check CHECK ((email ~ '^[a-zA-Z0-9.!#$%&''*+/=?^_`{|}~-]+@[a-zA-Z0-9](?:[a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?(?:\.[a-zA-Z0-9](?:[a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?)*$'::text))
);


Insert data into `tbl_departments`:

In [ ]:
%%sql
insert into tbl_departments (deptno, dname, loc, description)
values (10, 'ACCOUNTING', 'NEW YORK','preparation of financial statements, maintenance of general ledger, payment of bills, preparation of customer bills, payroll, and more.'), (20, 'RESEARCH', 'DALLAS','responsible for preparing the substance of a research report or security recommendation.'), (30, 'SALES', 'CHICAGO','division of a business that is responsible for selling products or services'), (40, 'OPERATIONS', 'BOSTON','administration of business practices to create the highest level of efficiency possible within an organization');

Insert data into `tbl_employees`:

In [2]:
%%sql

insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7369, 'SMITH', 'CLERK', 7902, '1980-12-17', 800, NULL, 20,'SMITH@acme.com', '{"skills":["accounting"]}');
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7499, 'ALLEN', 'SALESMAN', 7698, '1981-02-20', 1600, 300, 30,'ALLEN@acme.com', null);
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7521, 'WARD', 'SALESMAN', 7698, '1981-02-22', 1250, 500, 30,'WARD@compuserve.com', null);
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7566, 'JONES', 'MANAGER', 7839, '1981-04-02', 2975, NULL, 20,'JONES@gmail.com', null);
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7654, 'MARTIN', 'SALESMAN', 7698, '1981-09-28', 1250, 1400, 30,'MARTIN@acme.com', null);
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7698, 'BLAKE', 'MANAGER', 7839, '1981-05-01', 2850, NULL, 30,'BLAKE@hotmail.com', null);
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7782, 'CLARK', 'MANAGER', 7839, '1981-06-09', 2450, NULL, 10,'CLARK@acme.com', '{"skills":["C","C++","SQL"]}')
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7788, 'SCOTT', 'ANALYST', 7566, '1982-12-09', 3000, NULL, 20,'SCOTT@acme.com', '{"cat":"tiger"}');
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES  (7839, 'KING', 'PRESIDENT', NULL, '1981-11-17', 5000, NULL, 10,'KING@aol.com', null); 
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7844, 'TURNER', 'SALESMAN', 7698, '1981-09-08', 1500, 0, 30,'TURNER@acme.com', null);
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7876, 'ADAMS', 'CLERK', 7788, '1983-01-12', 1100, NULL, 20,'ADAMS@acme.org', null); 
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7900, 'JAMES', 'CLERK', 7698, '1981-12-03', 950, NULL, 30,'JAMES@acme.org', null);
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7902, 'FORD', 'DEVELOPER', 7566, '1981-12-03', 3000, NULL, 20,'FORD@acme.com', '{"skills":["YSQL","YCQL","SQL","CQL"]}'); 
insert into tbl_employees (empno, ename, job, mgr, hiredate, sal, comm, deptno, email, other_info) VALUES (7934, 'MILLER', 'CLERK', 7782, '1982-01-23', 1300, NULL, 10,'MILLER@acme.com', null);

UsageError: Cell magic `%%sql` not found.



Learn to use YSQL to update, access, and analyze the data stored in a database.

### Scenario 2: SQL Updates

Update salary for all employees who are not managers

Description:
A SQL update can compute the new value and return it without the need to query again. The following adds 100 to the salaries of all employees who are not managers and show the new value

In [ ]:
%%sql /* row counts */

update tbl_employees set sal=sal+100
where job != 'MANAGER'
returning ename,sal as new_salary;

### Scenario 3: Join

List all employees earning more than their managers using a self-join query.

Description:
A self join is a regular join, but the table is joined with itself. The following SQL statement matches employees with their manager and filters those that are earning more than their manager.

In [ ]:
%%sql

select 
  employee.ename,
  employee.sal,
  manager.ename as "manager ename",
  manager.sal as "manager sal"
from
  tbl_employees employee
join tbl_employees manager ON
  employee.mgr = manager.empno
where 1=1 
 and manager.sal<employee.sal
order by employee.sal;

### Scenario 4: Prepared Statements

Get salary for an employee with a prepared statement

Description:
Get salary for an employee with a prepared statement that declares a fixed query with typed parameters to prevent SQL injection


Step 1: 
Prepare a statement employee_salary with a parameterized query

A prepared statement declares parameterized SQL. The following will display the name and salary for a provided employee number (as integer).

In [ ]:
#%% python, but prepared statements as sql magic TODO make as fn_util
a = %sql prepare employee_salary(int) as select ename,sal from tbl_employees where empno=$1;

print (a)

In [ ]:
%%sql

prepare employee_salary(int) as select ename,sal from tbl_employees where empno=$1;

Step 2:
Use the EXECUTE statement to execute a previously prepared statement. The following SQL executes the prepared statement with an employee ID 7900

Once prepared, the statement can be called by providing only the parameter value. This displays the name and salary of employee number 7900

In [ ]:
%%sql

execute employee_salary(7900);

Step 3: Execute the same prepared statement another employee ID 7902

Without writing the SQL again, this displays the name and salary of employee number 7902

In [ ]:
%%sql

execute employee_salary(7902);

A prepared statement stays in the session and can be deallocated when not used anymore. The following frees the memory used by this statement:

In [ ]:
%%sql
deallocate employee_salary;

### Scenario 5: Indexes

Create and analyze index on the fly

Description: 
Create a new table and a specific index to avoid table scan and sorts


Step 1: Create a new demo table with randomly generated rows

`GENERATE_SERIES()` is a built-in function can generate rows. The following uses it to create a table with 42 rows and a random value from 1 to 10

In [ ]:
%%sql 

drop table if exists tbl_demo;

create table if not exists tbl_demo as select generate_series(1,42) num, round(10*random()) val;

Step 2: Create the index “demo_val” on demo table

With the goal to query, for a value, with numbers ordered, in the most efficient way, the following creates an index on “val” (hashed for distribution) and “num” in ascending order

In [ ]:
%%sql

create index idx_demo_val on tbl_demo (val hash, num asc);

Step 3: Gather optimizer statistics on table demo

The query planner chooses the best access path when provided with statistics about the data stored in the table. The following gathers those statistics.

In [ ]:
%%sql 

analyze tbl_demo;

Step 4: Query the Top-3 numbers for a specific value

The following displays the Top-3 numbers for the value 5

In [ ]:
%%sql

select * 
from tbl_demo 
where 1=1 
and val=5 
order by num asc 
fetch first 3 rows only;

Step 5: Verify that index is leading to faster query execution using explain analyze

When defining an index for a specific access pattern, it is good that the developer verifies that the index is used. The following shows that an “Index Only Scan” was used, without the need for an additional “Sort” operation

In [ ]:
%%sql

explain analyze select * from demo where val=5 order by num fetch first 3 rows only;

Step 6: Clean up the table for this exercise.

To leave the database in the same state as before this exercise, the following removes the demo table created before

In [ ]:
%%sql

drop table if exists demo;

## q2 | Built-in Functions
Learn powerful functions for performing complex database operations with ease

### Scenario 1: Window Functions
Compare employees hiring time interval by department using LAG function

LAG is a window function that provides access to the row before the current one. The following SQL statement uses WINDOW to define groups of employees by department, in order of their hiring date. LAG is used to access the previous row in this group, to compare the hiring date interval between two employees. FORMAT builds a text from column values, and COALESCE handles the first hire for which there is no previous row in the group. Without those window functions, this query would have required reading the same table two times.


In [ ]:
%%sql

select
dname,ename,job,
coalesce (
  'hired '||to_char(hiredate -
    lag(hiredate) over (per_dept_hiredate),'999')||' days after '||
    lag(ename) over (per_dept_hiredate),
    format('(1st hire in %L)',dname)
) as "last hire in dept"
from tbl_employees join tbl_departments using(deptno)
window per_dept_hiredate
  as (partition by dname order by hiredate)
order by dname,hiredate;

### Scenario 2. Regexp Matching

List all employees with `@gmail` or `.org` in their email addresses

Description: `REGEXP` performs a pattern match of a string expression. The following lists employees with an e-mail ending in `.org` or a domain starting with `gmail`.

In [ ]:
%%sql

select * 
from tbl_employees
where 1=1
and email ~ any ( ARRAY[ '@.*\.org$' , '@gmail\.' ] );

### Scenario 3. Arithmetic Date Intervals

The interval data type allows you to store and manipulate a period of time in years, months, days. 

The following example compares overlapping evaluation periods. A WITH clause defines the evaluation period length depending on the job.

Find employees with overlapping evaluation periods.

In [ ]:
%%sql

with emp_evaluation_period as (
 select ename,deptno,hiredate,
 hiredate + case when job in ('MANAGER','PRESIDENT')
 then interval '3 month' else interval '4 weeks'
 end evaluation_end from tbl_employees)
select * from tbl_employees_evaluation_period e1
 join emp_evaluation_period e2
 on (e1.ename>e2.ename) and (e1.deptno=e2.deptno)
where (e1.hiredate,e1.evaluation_end)
 overlaps (e2.hiredate,e2.evaluation_end);

### Scenario 4. CROSSTABVIEW

`CROSSTABVIEW` is a client command to display rows as columns. The following sums the salaries across jobs and departments and displays them as a cross-table

Display total salary per job and department as a cross-table using bash

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" #\d+
YB_PATH=${1}
DB_NAME=${2}

SQL_CROSSTABVIEW="select job, dname, sum(sal) from tbl_employees join tbl_departments using(deptno) group by dname, job \crosstabview"

cd $YB_PATH

./bin/ysqlsh -d ${DB_NAME} -c ${SQL_CROSSTABVIEW}

In [ ]:
%%sql

select job, dname, sum(sal)
from tbl_employees join tbl_departments using(deptno)
group by dname, job

### Scenario 5: NTILE Function

Split e-mails in 3 groups and format them

Description: In order to send e-mails to all employees in different batches, you will split them into 3 groups using the NTILE function, and format them with the FORMAT function and aggregate them in a comma-separated list with the STRING_AGG function

In [ ]:
%%sql

with groups as (
 select ntile(3) over (order by empno) group_num
 ,* 
 from tbl_employees
)
select string_agg(format('<%s> %s',ename,email),', ') 
from groups group by group_num;

## q3 | Advanced Features
Expand your YSQL skills by completing the following 5 scenarios

### Scenario 1:  GIN Index on Document
List employees that know SQL

Description: The skills are stored in the semi-structured JSON document. We can query them with @>, ?, ?& and ?| operators. And, for best performance, index them.

Step 1: Create GIN index on JSON document

GIN indexes can provide fast access to elements inside a JSON document. The following creates an index on the ‘skills’ attributes within ‘other_info’ JSON column

In [ ]:
%%sql 

create index idx_emp_skills_ybgin on tbl_employees using gin((other_info->'skills'));

Step 2: Query the JSON attribute list

SQL queries can navigate into the JSON document with -> and check if an array contains a value with @>. The following searches the employees with the “SQL” skill

In [ ]:
%%sql 

select * from tbl_employees where other_info->'skills' @> '"SQL"' ;

Step 3: Use the Explain plan to verify that index usage

Thanks to the GIN index, this search doesn’t need to read all documents. The following shows the execution plan being an indexed access path.

In [ ]:
%%sql
-- TODO CHECK syntax, spacing
explain select * from tbl_employees where other_info->'skills' @> '"SQL"' ;

### Scenario 2: Text Search
Build a search index on department descriptions

Description: 
SQL queries can search in the text for some words using the `to_tsvector()` function to extract a list of words that can be compared. We will find all department descriptions with the words 'responsible' and 'services' in it

Step 1: Create text search index on the description column

GIN indexes can provide fast access to words inside a text. The following creates an index for the simple-grammar vector of words extracted from the department description.

In [ ]:
%%sql

create index idx_dept_description_ybgin on tbl_departments using gin (( to_tsvector('simple',description) ));

Step 2: Query on description for matching words

The following compares the simple-grammar vector of words extracted from the department description with a word search pattern to find the departments which contain ‘responsible’ and ‘services’ in their description.

In [ ]:
%%sql

select * from tbl_departments where to_tsvector('simple',description) @@ to_tsquery('simple','responsible & services');

Step 3: Explain plan to verify that index is used

Thanks to the GIN index, this search doesn’t need to read all rows and text. The following shows the execution plan being an indexed access path

In [ ]:
%%sql

explain select * from tbl_departments where to_tsvector('simple',description) @@ to_tsquery('simple','responsible & services');

### Scenario 3. Stored Procedures
Transfer commission from one employee to another


Description: 
A stored procedure can encapsulate a procedural logic into an atomic operation. We will create one in PL/pgSQL, named "commission_transfer", that transfers commission “amount” from “empno1” to “empno2”


Step 1: Create the procedure for the commission transfer between employees

The procedure has two SQL operations: 
- decrease from `empno1`
- add to `empno2`


In addition, the procedure must check for an error and raise a custom exception if `empno1` does not have the amount to be transferred

In [ ]:
%%sql

create or replace procedure commission_transfer(empno1 int, empno2 int, amount int) 
as $$
begin
  update tbl_employees set comm=comm-commission_transfer.amount
    where empno=commission_transfer.empno1 and comm>commission_transfer.amount;
  if not found then 
    raise exception 'Cannot transfer % from %',amount,empno1;
  end if;
  update tbl_employees set comm=comm+commission_transfer.amount
    where emp.empno=commission_transfer.empno2;
  if not found then 
    raise exception 'Cannot transfer from %',empno2; 
  end if;
end;
$$ language plpgsql;

Step 2: Call the procedure with employee IDs and the amount to be transferred

Once defined, the stored procedure is called with values for all parameters. This transfers 100 from tbl_employeesloyee 7521 to 7654

In [ ]:
%%sql

call commission_transfer(7521,7654,100);

Step 3: List all employees who have received commission

The following displays all employees having a commission, to verify that 100 have been transferred

In [ ]:
%%sql

select * from tbl_employees where comm is not null;

Step 4: Call the procedure with employee IDs and amount that is not allowed to invoke error handling

The following attempts to transfer 1000000, more than what 7521 has. It raises the “Cannot transfer” error defined in the procedure and automatically reverts all intermediate changes to return to a consistent state

In [ ]:
%%sql

call commission_transfer(7521,7654,1000000);

### Scenario 4. Triggers
Record the last update time of each row automatically

Description
We will add a column to hold the last update time, and declare a trigger to update it automatically, on the departments table

SQL Statement
Step 1: Add a column to store the last update time

The structure of a SQL table can evolve. With the goal of recording the last update, the following adds a “last_update” column to the department table.

In [ ]:
%%sql

alter table tbl_departments add last_update timestamptz;

Step 2: Add a function `fn_dept_last_update` to set the last update time. The following SQL query uses the built-in function transaction_timestamp(), which returns the current date and time at the start of the current transaction

A stored function declares some procedural logic that returns a value. The following returns the “new” state for a trigger after setting the “last_update” to the current time.

In [ ]:
%%sql

create or replace function fn_dept_last_update() returns trigger as $$
begin
  new.last_update:=transaction_timestamp();
  return new;
end;
$$ language plpgsql;

Step 3: Create a trigger `trg_dept_last_update` to call the function `fn_dept_last_update()` on each table update.

The previous function can be called automatically. The following trigger executes it on each row update for the departments table.

In [ ]:
%%sql

create trigger trg_dept_last_update
before update on tbl_departments
for each row
execute procedure fn_dept_last_update();

Step 4: Display the current state of the table

In order to verify the automatic logging of the last update time, the following displays the current state of departments before any update

In [3]:
%%sql

select deptno,dname,loc,last_update 
from tbl_departments;

UsageError: Cell magic `%%sql` not found.


Step 5: Update multiple rows in one transaction in the dept table

Multiple updates can be declared in a single atomic transaction with BEGIN TRANSACTION and COMMIT. The following updates the location of departments 30 and 40 with a 3 seconds interval in between.


In [ ]:
%%sql
begin transaction;
update tbl_departments set loc='SUNNYVALE' where deptno=30;
select pg_sleep(3);
update tbl_departments set loc='SUNNYVALE' where deptno=40;
commit;

Step 6: Display the new state of the table to see the last_update column being updated

The following shows that in addition of the change location, the last update timestamp has been automatically set. Even if the updates were done at 3 seconds interval, they show the same update time because all was run in an atomic transaction


In [ ]:
%%sql
select deptno,dname,loc,last_update from tbl_departments;

### Scenario 5. Materialized Views
View
Pre-compute analytics for reporting, with a materialized view

Description
In order to get fast on-demand reports, we create a materialized view to store pre-joined and pre-aggregated data. This view will store the total salary per department, the number of employees, and the list of jobs in the department.

SQL Statement
Step 1: Create the materialized view

In [ ]:
%%sql

create materialized view mvw_report_sal_per_dept as
select 
deptno,dname,
sum(sal) sal_per_dept,
count(*) num_of_employees,
string_agg(distinct job,', ') distinct_jobs
from tbl_departments join tbl_employees using(deptno)
group by deptno,dname
order by deptno;

Step 2: Indexes can be created on it. This one allows fast queries on a range of total salary

In [ ]:
%%sql
drop index if exists idx_mvw_report_sal_per_dept_sal;
create index if not exists idx_mvw_report_sal_per_dept_sal on report_sal_per_dept(sal_per_dept desc);

Step 3: A refresh can be scheduled on a daily basis to re-compute it in the background with a simple command.

In [ ]:
%%sql

refresh materialized view mvw_report_sal_per_dept;

Step 4: Querying the materialized view will show the data consistent as of the last refresh. This one lists the departments with a total salary lower than 10000

In [ ]:
%%sql

select *
from mvw_report_sal_per_dept
where 1=1
and sal_per_dept<=10000
order by sal_per_dept;

Step 5: The execution plan shows that no additional join or group by are needed when querying this materialized view:

In [ ]:
%%sql
explain analyze
select *
from mvw_report_sal_per_dept
where 1=1
and sal_per_dept<=10000
order by sal_per_dept;

---
# All done!
In this lab, you completed the following:

- Setup
  - Created the `db_ybu` database with `ysqlsh`
  - Created utils
  - Created tables and loaded data using DDL and DML scripts
  - Connected to the database using a PostgreSQL driver for Python

Next, run the following cell to open `02_Demystifying_table_sharding_tablets_and_data_distribution.ipynb`.